In [6]:
search_for = 105
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.017091035842895508
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 264697301
type: [1, 1, 1, 1, 105] 105
cases of this type: 121550625
10000 43.04818748762079
20000 45.4325052913093
30000 46.06622365022217
40000 157.4163438152754
50000 196.29699888766652
60000 190.4095522175311
70000 169.15116898105254
80000 171.78117377156175
90000 173.42187388139425
100000 183.3175571474995
110000 182.73862639752204
120000 180.97690032267093
130000 176.25665157534297
140000 172.35489056516363
150000 178.3060034751371
160000 175.71810232780663
170000 180.55314078401005
180000 175.63793256230656
190000 167.20973229126386
200000 176.07875895326598
210000 166.95623477753648
220000 179.49107819563648
230000 172.6098398892227
240000 179.32696938931144
250000 176.94074155724692
260000 170.09487056321947
270000 177.0276089514674
280000 182.23094125173753
290000 190.14714644619318
300000 206.2267090885794
310000 181.

3040000 153.26083148564925
3050000 155.35566340729616
3060000 163.22357936726382
3070000 163.36661372620537
3080000 151.1933542090833
3090000 154.2562343989086
3100000 154.969894916453
3110000 155.24750121311789
3120000 167.8134357305517
3130000 165.22522646983637
3140000 155.7161299239111
3150000 162.38402184325514
3160000 167.93761040305137
3170000 166.29720373687255
3180000 171.15628967790124
3190000 166.44708698873617
3200000 180.51114899931784
3210000 173.1199367977011
3220000 161.9410646945787
3230000 167.78187254001418
3240000 164.75728366037634
3250000 172.54251286534353
3260000 167.6508422915758
3270000 182.32245047601353
3280000 199.8746766955995
3290000 175.06563287324235
3300000 165.52444950900232
3310000 161.40437518549467
3320000 165.45182538436913
3330000 169.36293042483692
3340000 180.79661401700452
3350000 172.31242766445092
3360000 165.48983766826518
3370000 164.6107587903845
3380000 164.11303699014027
3390000 164.9926735595775
3400000 163.36353637955858
3410000 167.8

6120000 162.54639082584094
6130000 176.38158839649387
6140000 159.50192540161194
6150000 158.0548378242887
6160000 161.2832544262584
6170000 162.34255923733232
6180000 152.20710259657847
6190000 161.51337055914848
6200000 156.24078792809829
6210000 156.39335924938806
6220000 158.8704174509432
6230000 166.38063373289575
6240000 167.65384435835807
6250000 169.5749375619508
6260000 174.69303263789482
6270000 162.86509244670518
6280000 167.0477175465166
6290000 165.02477198889585
6300000 166.05768937211647
6310000 148.09992451854953
6320000 146.69008248096478
6330000 159.2259036918538
6340000 185.48181179789253
6350000 22.515288311491393
6360000 22.63171612636286
6370000 22.415213385192693
6380000 22.556365075228854
6390000 22.54279162231493
6400000 118.28261384697706
6410000 189.6179919659271
6420000 153.7750522244168
6430000 142.44981279947766
6440000 148.2444424296
6450000 160.34286950093625
6460000 168.08422888778043
6470000 169.95101998166294
6480000 162.24319338434603
6490000 165.702

9200000 56.86528860228395
9210000 120.20897788055952
9220000 191.9987066808922
9230000 163.68554267331214
9240000 147.09169342950398
9250000 175.1877153859085
9260000 172.73750925536203
9270000 168.9979172196555
9280000 179.18713304127445
9290000 163.33907919977491
9300000 156.15869551940364
9310000 176.3883928950244
9320000 184.74110571029473
9330000 152.5623659884896
9340000 44.93600000146852
9350000 45.34395788878015
9360000 45.10929993278584
9370000 44.4404675224974
9380000 44.47527917126173
9390000 160.70054037667649
9400000 175.4697068152866
9410000 164.6344838902928
9420000 164.633918310875
9430000 158.7925258978425
9440000 164.26078566530776
9450000 168.1218806681152
9460000 162.83811826070837
9470000 156.34285378222364
9480000 164.92795975109868
9490000 167.4599436012135
9500000 161.0389333547638
9510000 166.44252141090462
9520000 164.83213234538232
9530000 155.12615336256036
9540000 156.23615247946898
9550000 161.38617978946874
9560000 157.17517272060417
9570000 158.293673070

12200000 176.10183843869493
12210000 162.66659630915046
12220000 146.45038935300624
12230000 139.02540651186922
12240000 136.39964471875066
12250000 150.26228343338812
12260000 150.57166437706672
12270000 148.765550212413
12280000 151.62503180535663
12290000 150.36738543621559
12300000 145.15834929738628
12310000 155.85899692106554
12320000 156.75509390801577
12330000 147.26560986094677
12340000 154.38598999297548
12350000 155.7139584357471
12360000 158.05643990253117
12370000 151.25890373850797
12380000 172.04932510189715
12390000 172.65625687452132
12400000 163.35989372314455
12410000 165.44869993225998
12420000 154.71039076981378
12430000 158.83804528318265
12440000 153.93096213619208
12450000 155.9640937865246
12460000 159.96581507932376
12470000 160.18364062202735
12480000 149.597151417451
12490000 165.81835531084485
12500000 168.52836053316318
12510000 165.2364277072103
12520000 164.6477290001048
12530000 161.634692253721
12540000 164.70209601949236
12550000 154.28880450907965
12

...

253590000 2.888880190056125
253600000 3.9805384860848325
253610000 4.591718839249829
253620000 3.743840887041099
253630000 2.594195153540936
253640000 2.12094834231925
253650000 3.5621616974113124
253660000 1.8505667121677598
253670000 4.042531391585469
253680000 4.275120567869657
253690000 4.275099224261204
253700000 3.2594832370592353
253710000 0.9988571086365117
253720000 3.22237807100049
253730000 3.8992050736165047
253740000 4.26234232401765
253750000 3.803311426196926
253760000 2.077882318076233
253770000 3.7468195895563694
253780000 4.633431225831204
253790000 1.7523904768763183
253800000 3.4204498159684205
253810000 4.228418373229987
253820000 3.8879493477879166
253830000 2.5456478686830994
253840000 0.8913844191179342
253850000 3.0767402432320514
253860000 3.8557340531966817
253870000 4.148084932258381
253880000 2.8583901945536336
253890000 2.1614338035359646
253900000 4.878149749179847
253910000 4.397159849933266
253920000 1.804111577718344
253930000 2.508775217774437
2539400

256460000 3.385862069764376
256470000 3.594425529304352
256480000 3.7770719552890064
256490000 3.3056482862778305
256500000 2.501250854597105
256510000 1.6307536650181016
256520000 2.6173064171910885
256530000 2.536535290253871
256540000 0.7220153804149694
256550000 2.461548236569961
256560000 2.488578181903945
256570000 1.3145785466684368
256580000 2.6264711082870167
256590000 3.2766768095326158
256600000 3.8300931669931275
256610000 3.738759112443924
256620000 3.1034253172478277
256630000 1.7196666858106586
256640000 1.5439138081415495
256650000 3.075732771465427
256660000 2.013341905905114
256670000 0.6932171210470199
256680000 2.8021924531782862
256690000 2.582074072126415
256700000 1.2533692347826957
256710000 2.202223086528341
256720000 3.0630616123408543
256730000 3.9655929837819537
256740000 3.334351243219614
256750000 2.722907704849064
256760000 1.5282318219228586
256770000 2.321893700169033
256780000 2.893279583539115
256790000 1.6304980631722212
256800000 0.6860366662369504


259270000 0.6004466917975015
259280000 0.5733185644431313
259290000 0.7040000537262293
259300000 0.7620619376437664
best so far: 0
type: [1, 3, 1, 35, 1] 105
cases of this type: 128625
259310000 0.3951393638156652
259320000 0.37392998286139306
259330000 0.3698059497066405
259340000 0.363252426632444
259350000 0.3439375575401452
259360000 0.3462634824360013
259370000 0.34967676530746616
259380000 0.3634631802779635
259390000 0.36807157494022447
259400000 0.3498762044622898
259410000 0.3360332361953126
259420000 0.3382535727690525
best so far: 0
type: [1, 3, 5, 1, 7] 105
cases of this type: 180075
259430000 0.36728849947893616
259440000 0.8146266388130983
259450000 0.8074371637238265
259460000 0.5979247660151918
259470000 0.3851730355106725
259480000 0.3831394194279379
259490000 0.37140828214448696
259500000 0.3822652702741358
259510000 0.41169297016061673
259520000 0.4059311337530613
259530000 0.40777310939918626
259540000 0.39902841114278637
259550000 0.38056388641170663
259560000 0.37

261720000 0.6531782994626893
261730000 0.48358365717516344
best so far: 0
type: [1, 7, 1, 3, 5] 105
cases of this type: 118125
261740000 0.6530499396169185
261750000 0.23262016315859554
261760000 0.2542902341316011
261770000 0.24364740099793672
261780000 0.2565712069287896
261790000 0.2564897389509347
261800000 0.2538499817210833
261810000 0.24761800477628576
261820000 0.2427028256042401
261830000 0.2361170791040659
261840000 0.23937916341467036
261850000 0.2374749290712012
best so far: 0
type: [1, 7, 1, 5, 3] 105
cases of this type: 70875
261860000 0.23974196099666756
261870000 0.24218353466168377
261880000 0.24367254841246208
261890000 0.2515723765280644
261900000 0.21573796669692466
261910000 0.2309131366195414
261920000 0.2324911267039776
best so far: 0
type: [1, 7, 1, 15, 1] 105
cases of this type: 23625
261930000 0.227957821679016
261940000 0.183763246103247
best so far: 0
type: [1, 7, 3, 1, 5] 105
cases of this type: 39375
261950000 0.18213639880971114
261960000 0.21308062044547

264040000 0.07394010182482004
264050000 0.07903369162925084
264060000 0.08621429021124045
264070000 0.07220096559647718
264080000 0.0659271212233901
264090000 0.07102049076506164
best so far: 0
type: [3, 1, 1, 35, 1] 105
cases of this type: 42875
264100000 0.053352328043003876
264110000 0.03854164290525516
264120000 0.03657182449883222
264130000 0.03728948871830437
best so far: 0
type: [3, 1, 5, 1, 7] 105
cases of this type: 60025
264140000 0.04313630554501216
264150000 0.06806126049114598
264160000 0.040462945416861115
264170000 0.04022650370687246
264180000 0.04102272280599673
264190000 0.03925568122337924
best so far: 0
type: [3, 1, 5, 7, 1] 105
cases of this type: 8575
264200000 0.0748313903760314
best so far: 0
type: [3, 1, 7, 1, 5] 105
cases of this type: 30625
264210000 0.03561050962648127
264220000 0.037541473773492706
264230000 0.03715686965570847
best so far: 0
type: [3, 1, 7, 5, 1] 105
cases of this type: 6125
264240000 0.03334515835606389
best so far: 0
type: [3, 1, 35, 1, 